In [7]:
import cudf as pd
import pandas 
import numpy as np
import time as TIME
import datetime as dt
from datetime import date
from datetime import timedelta
import math
import os

import warnings
warnings.filterwarnings("ignore")

In [4]:
df=pd.read_csv('fno_stocks_csv.csv', header= 1)
Tickers = (df["Symbol"])
Tickers = Tickers.astype(str)

#Tickers = Tickers.drop(Tickers[Tickers.values == "BANKNIFTY"].index.values[0], axis =0)
#Tickers = Tickers.drop(Tickers[Tickers.values == "NIFTY"].index.values[0], axis =0)
#Tickers = Tickers.reset_index(drop = True)
Tickers

0            IDEA
1      COROMANDEL
2            ZEEL
3       NESTLEIND
4        ASHOKLEY
          ...    
167    APOLLOHOSP
168        INDIGO
169    CANFINHOME
170    ICICIPRULI
171       SBILIFE
Name: Symbol, Length: 172, dtype: object

In [8]:
%%time
weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
Daily_pct_change = pd.DataFrame()
current_date = date(2017,1,1)
current_date = pandas.Timestamp(current_date, tz =19800)
#todays_date = dt.datetime.now().date()
todays_date  = date(2021,9,23)
flag = 0
#Loop from start date till today
while(current_date <= todays_date):
    #Calculating daily pct change for all stocks till 10 Am
    
    start = dt.datetime.now()
    pct_change = {}
    Daily_pct_change = pd.read_csv("Day_Wise_Stats.csv", index_col = 0,  parse_dates = [2])
    if(flag==0):
        current_date = Daily_pct_change.Date.iloc[-1] + timedelta(days =1)
        flag = 1
        if(weekDays[current_date.weekday()] == "Saturday" or weekDays[current_date.weekday()] == "Sunday"):
            print(str(current_date) + " : Started")
            print(str(current_date) + " : Finished") 
            current_date = current_date + timedelta(days =1)
            if(weekDays[current_date.weekday()] == "Saturday" or weekDays[current_date.weekday()] == "Sunday"):
                print(str(current_date) + " : Started")
                print(str(current_date) + " : Finished") 
                current_date = current_date + timedelta(days =1)
                                                                     
    print(str(current_date) + " : Started")
    for ticker in Tickers:
        daily_data = pd.read_csv("Historical_Data/" + ticker + ".csv", index_col = 0, parse_dates = [1])
        daily_data = daily_data[daily_data.date <= pandas.Timestamp(current_date) + timedelta(days =1, minutes = 555)]
        daily_data = daily_data[daily_data.date >= pandas.Timestamp(current_date)]
        daily_data = daily_data.reset_index(drop = True)
        if(daily_data.empty == True):
            continue
        change_value = (daily_data[daily_data["date"] == daily_data["date"].iloc[0] + timedelta(minutes = 60)].close.values[0] - daily_data.close[0])/daily_data.close[0] 
        pct_change[ticker] = change_value
        #rint(change_value)
        
    top_gainers = dict(sorted(pct_change.items(), key=lambda x: x[1], reverse=True))
    change = pd.DataFrame()
    change["Tickers"] = top_gainers.keys()
    change["PCT_Change"] = top_gainers.values()
    if(change.empty == True):
        print(str(current_date) + " : Finished") 
        continue
        
    gain_number = change[change["PCT_Change"] > 0].shape[0] + 1 
    loss_number = change[change["PCT_Change"] <= 0].shape[0] + 1
    ad_ratio    = gain_number/loss_number
    end = dt.datetime.now() - start
    print("Time Required to calculate pct change: " + str(end)) 
    print("PCT Change Calculated!")
    for i in range(5):
        print(change["Tickers"][i])
    new_row = {"Date": current_date, "Stock1": change["Tickers"][0], "Stock2" : change["Tickers"][1], "Stock3": change["Tickers"][2], "Stock4": change["Tickers"][3],"Stock5": change["Tickers"][4] , "AD_Ratio":ad_ratio }
    Daily_pct_change = Daily_pct_change.append(new_row, ignore_index = True )
    Daily_pct_change.to_csv("Day_Wise_Stats.csv")
    
    date_data = pd.read_csv("Historical_Data/" + change["Tickers"][0] + ".csv", index_col = 0, parse_dates = [1])
    #date_data = date_data[date_data.date <= pandas.Timestamp(current_date) + timedelta(days =1, minutes = 555)]
    date_data = date_data[date_data.date >= pandas.Timestamp(current_date)]
    date_data = date_data.reset_index(drop = True)
    date_data = date_data[:76]
    print(str(current_date) + " : Finished") 
    current_date = date_data.date.iloc[-1]
       

TypeError: can only concatenate str (not "datetime.timedelta") to str

In [6]:
change

,Tickers,PCT_Change
0,INDUSINDBK,0.010211
1,DLF,0.009711
2,MANAPPURAM,0.009671
3,L&TFH,0.009287
4,HDFC,0.008431
...,...,...
154,M&M,-0.009894
155,TCS,-0.010714
156,PNB,-0.013928
157,ABFRL,-0.014217


In [13]:
current_date = date(2017,1,2)
current_date = pd.Timestamp(current_date, tz =19800)
Daily_pct_change = pd.DataFrame()
#todays_date = dt.datetime.now().date()
todays_date  = date(2021,9,23)
pct_change = {}
start = dt.datetime.now()
print(str(current_date) + " : Started")
for ticker in Tickers:
    daily_data = pd.read_csv("Historical_Data/" + ticker + ".csv", index_col = 0, parse_dates = [2])
    daily_data = daily_data[daily_data.date <= pd.Timestamp(current_date) + timedelta(days =1, minutes = 555)]
    daily_data = daily_data[daily_data.date >= pd.Timestamp(current_date)]
    daily_data = daily_data.reset_index(drop = True)
    if(daily_data.empty == True):
        continue
    change_value = (daily_data[daily_data["date"] == daily_data["date"].iloc[0] + timedelta(minutes = 60)].close.values[0] - daily_data.close[0])/daily_data.close[0] 
    pct_change[ticker] = change_value
    #rint(change_value)

top_gainers = dict(sorted(pct_change.items(), key=lambda x: x[1], reverse=True))
change = pd.DataFrame()
change["Tickers"] = top_gainers.keys()
change["PCT_Change"] = top_gainers.values()
gain_number = change[change["PCT_Change"] > 0].shape[0] + 1 
loss_number = change[change["PCT_Change"] <= 0].shape[0] + 1
ad_ratio    = gain_number/loss_number
end = dt.datetime.now() - start
print("Time Required to calculate pct change: " + str(end)) 
print("PCT Change Calculated!")
for i in range(5):
    print(change["Tickers"][i])
    
new_row = {"Date": current_date, "Stock1": change["Tickers"][0], "Stock2" : change["Tickers"][1], "Stock3": change["Tickers"][2], "Stock4": change["Tickers"][3],"Stock5": change["Tickers"][4] , "AD_Ratio":ad_ratio }
Daily_pct_change = Daily_pct_change.append(new_row, ignore_index = True )
Daily_pct_change.to_csv("Day_Wise_Stats.csv")

date_data = pd.read_csv("Historical_Data/" + change["Tickers"][0] + ".csv", index_col = 0, parse_dates = [1])
date_data = date_data[date_data.date <= pd.Timestamp(current_date) + timedelta(days =1, minutes = 555)]
date_data = date_data[date_data.date >= pd.Timestamp(current_date)]
date_data = date_data.reset_index(drop = True)
current_date = date_data.date.iloc[-1]
print(str(current_date) + " : Finished")


2017-01-02 00:00:00+05:30 : Started
Time Required to calculate pct change: 0:03:44.616232
PCT Change Calculated!
BHARATFORG
ASTRAL
MOTHERSUMI
PETRONET
BHEL


NameError: name 'new_row_daily' is not defined